In [25]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, recall_score, accuracy_score
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import tabulate

In [24]:
!!pip install tabulate

['Requirement already satisfied: tabulate in c:\\users\\vasuk\\appdata\\local\\programs\\python\\python310\\lib\\site-packages (0.9.0)',
 '',
 '[notice] A new release of pip available: 22.3.1 -> 23.0.1',
 '[notice] To update, run: python.exe -m pip install --upgrade pip']

In [76]:
import pickle

#Load the trained Best Linear Regression model
lr=pickle.load(open('Linear_class.sav', 'rb'))
# load Logistic Regression model
log = pickle.load(open('best_log.sav','rb'))
#load the best knn model
knn = pickle.load(open('knn_class.sav', 'rb'))
#load Random fores model
rf = pickle.load(open('RF_class.sav', 'rb'))
#load Decision Trees model
dt = pickle.load(open('Tree_class.sav','rb'))


In [38]:
df = pd.read_csv('Group_9_data_cleaned.csv')
# removing a redundant column
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

,current_1,current_2,current_3,current_4,current_5,current_6,current_7,current_8,current_9,current_10,...,current_40,current_41,current_42,current_43,current_44,current_45,current_46,current_47,current_48,output
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


In [39]:
# Dividing the data into differnt sets
X = df.drop(['output'],axis=1)
y = df['output']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=100)

# Linear regression and KNN uses scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
model=scaler.fit(X)
scaled_X=model.transform(X)
scaled_X 

# Decision Trees and Random Forest does not need scaling 
x_train_scaled,x_test_scaled,y_train,y_test = train_test_split(scaled_X, y, test_size = 0.3, random_state=100)
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.3, random_state=100)


In [28]:
from tabulate import tabulate


## Linear Regression Testing data  Results 

In [46]:
predictions = np.zeros((x_test_scaled.shape[0], len(lr)))
for i, model in enumerate(lr):
    predictions[:, i] = model.predict(x_test_scaled)
# assign class based on highest prediction score
y_test_pred = np.argmax(predictions, axis=1)    

# Roc auc score 
test_predictions = np.zeros((x_test_scaled.shape[0], len(lr)))
for i, model in enumerate(lr):
    test_predictions[:, i] = model.predict(x_test_scaled)
test_roc_auc_scores = []
for c in np.unique(y):
    y_binary_test = (y_test == c).astype(int)
    if c <= i:
       y_pred = test_predictions[:, c]
       roc_auc = roc_auc_score(y_binary_test, y_pred)
       test_roc_auc_scores.append(roc_auc)
test_avg_roc_auc_score = np.mean(test_roc_auc_scores)

print(f" The accuracy for linear model is {accuracy_score(y_test,y_test_pred)}")
print(f" The Recall_score for linear model is {recall_score(y_test,y_test_pred,average = 'macro')}")
print(f" The F1_score for linear model is {f1_score(y_test,y_test_pred,average = 'weighted')}")
print(f" The roc auc score for linear model is {test_avg_roc_auc_score}")      

 The accuracy for linear model is 0.02643422776733322
 The Recall_score for linear model is 0.024629673626160805
 The F1_score for linear model is 0.027335919534643875
 The roc auc score for linear model is 0.5322959052292122


C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [47]:
# confusion matrix for linear model of test data
print(confusion_matrix(y_test,y_test_pred))


[[   0    0    0    0    0    0    0    0    0    0    0    0]
 [1553    0    0    0    0   47    0    1    3    0    0    0]
 [   0  827    3    2    0    8    0    0    4  554  287    0]
 [  56   98  237  218   43   40  626   19   10  192    2    0]
 [   0    9    2  980    9    0  493    2    3   63    0    0]
 [  45    1    5  276  504   10  334  259   82    8    2    0]
 [1141    6    0    0    2  456    0    9   31    0    6    0]
 [   0    0    0   21    0    0 1567    0    0    2    0    0]
 [ 318    2    1  111  137   14   52  805  144    0    1    0]
 [ 494    6    0    2    4   56    0   50  936    8   12    0]
 [   0  179    2    2    0    0    1    0    0 1379   72    0]
 [   0    9    0    0    0    0    0    0    8   20 1570    0]]


In [ ]:
# classification report for linear model of test data 

In [48]:
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00      1604
           2       0.01      0.00      0.00      1685
           3       0.14      0.14      0.14      1541
           4       0.01      0.01      0.01      1561
           5       0.02      0.01      0.01      1526
           6       0.00      0.00      0.00      1651
           7       0.00      0.00      0.00      1590
           8       0.12      0.09      0.10      1585
           9       0.00      0.01      0.00      1568
          10       0.04      0.04      0.04      1635
          11       0.00      0.00      0.00      1607

    accuracy                           0.03     17553
   macro avg       0.03      0.02      0.03     17553
weighted avg       0.03      0.03      0.03     17553



C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

## Logistic Regression Test data results

In [77]:
y_test_log = log.predict(x_test_scaled)
y_test_prob_log = log.predict_proba(x_test_scaled)
print(f" The accuracy for knn model is {accuracy_score(y_test,y_test_log)}")
print(f" The Recall_score for knn model is {recall_score(y_test,y_test_log,average = 'macro')}")
print(f" The F1_score for knn model is {f1_score(y_test,y_test_log,average = 'weighted')}")
print(f" The roc auc score for knn model is {roc_auc_score(y_test,y_test_prob_log,average='weighted',multi_class='ovr')}")      

 The accuracy for knn model is 0.9054292713496268
 The Recall_score for knn model is 0.9054255304241362
 The F1_score for knn model is 0.9048152927309002
 The roc auc score for knn model is 0.9943484272756621


In [78]:
# print confusion matrix of test data
print(confusion_matrix(y_test,y_test_pred))

[[   0    0    0    0    0    0    0    0    0    0    0    0]
 [1553    0    0    0    0   47    0    1    3    0    0    0]
 [   0  827    3    2    0    8    0    0    4  554  287    0]
 [  56   98  237  218   43   40  626   19   10  192    2    0]
 [   0    9    2  980    9    0  493    2    3   63    0    0]
 [  45    1    5  276  504   10  334  259   82    8    2    0]
 [1141    6    0    0    2  456    0    9   31    0    6    0]
 [   0    0    0   21    0    0 1567    0    0    2    0    0]
 [ 318    2    1  111  137   14   52  805  144    0    1    0]
 [ 494    6    0    2    4   56    0   50  936    8   12    0]
 [   0  179    2    2    0    0    1    0    0 1379   72    0]
 [   0    9    0    0    0    0    0    0    8   20 1570    0]]


In [79]:
# print classification report of test data
print(classification_report(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00      1604
           2       0.01      0.00      0.00      1685
           3       0.14      0.14      0.14      1541
           4       0.01      0.01      0.01      1561
           5       0.02      0.01      0.01      1526
           6       0.00      0.00      0.00      1651
           7       0.00      0.00      0.00      1590
           8       0.12      0.09      0.10      1585
           9       0.00      0.01      0.00      1568
          10       0.04      0.04      0.04      1635
          11       0.00      0.00      0.00      1607

    accuracy                           0.03     17553
   macro avg       0.03      0.02      0.03     17553
weighted avg       0.03      0.03      0.03     17553



C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

## KNN classification Test data results 

In [53]:
y_test_knn = knn.predict(x_test_scaled)
y_test_prob_knn = knn.predict_proba(x_test_scaled)
print(f" The accuracy for knn model is {accuracy_score(y_test,y_test_knn)}")
print(f" The Recall_score for knn model is {recall_score(y_test,y_test_knn,average = 'macro')}")
print(f" The F1_score for knn model is {f1_score(y_test,y_test_knn,average = 'weighted')}")
print(f" The roc auc score for knn model is {roc_auc_score(y_test,y_test_prob_knn,average='weighted',multi_class='ovr')}")      

 The accuracy for knn model is 0.9754457927419814
 The Recall_score for knn model is 0.975715100175092
 The F1_score for knn model is 0.9754475439861401
 The roc auc score for knn model is 0.9994078439670819


In [54]:
# confusion matrix for the test data
print(confusion_matrix(y_test,y_test_knn))

[[1538    0    2    0    0   64    0    0    0    0    0]
 [   1 1614    0    0    0    6    0    0    3   61    0]
 [   0    0 1539    0    0    2    0    0    0    0    0]
 [   0    0    0 1545   15    0    0    1    0    0    0]
 [   1    0    2   13 1478    0    0   32    0    0    0]
 [  42    4    0    0    3 1599    0    1    2    0    0]
 [   0    0    0    0    0    0 1590    0    0    0    0]
 [   0    0    0   28   28    2    0 1525    2    0    0]
 [   2    1    0    1    4    8    0    5 1547    0    0]
 [   0   92    0    0    0    3    0    0    0 1540    0]
 [   0    0    0    0    0    0    0    0    0    0 1607]]


In [55]:
# classification report for the test data
print(classification_report(y_test,y_test_knn))

              precision    recall  f1-score   support

           1       0.97      0.96      0.96      1604
           2       0.94      0.96      0.95      1685
           3       1.00      1.00      1.00      1541
           4       0.97      0.99      0.98      1561
           5       0.97      0.97      0.97      1526
           6       0.95      0.97      0.96      1651
           7       1.00      1.00      1.00      1590
           8       0.98      0.96      0.97      1585
           9       1.00      0.99      0.99      1568
          10       0.96      0.94      0.95      1635
          11       1.00      1.00      1.00      1607

    accuracy                           0.98     17553
   macro avg       0.98      0.98      0.98     17553
weighted avg       0.98      0.98      0.98     17553



## Random Forests test data results 

In [59]:
y_test_rf = rf.predict(X_test)
y_test_prob_rf = rf.predict_proba(X_test)
print(f" The accuracy for random forests model is {accuracy_score(y_test,y_test_rf)}")
print(f" The Recall_score for random forests model is {recall_score(y_test,y_test_rf,average = 'macro')}")
print(f" The F1_score for random forests model is {f1_score(y_test,y_test_rf,average = 'weighted')}")
print(f" The roc auc score for random forests model is {roc_auc_score(y_test,y_test_prob_rf,average='weighted',multi_class='ovr')}")      

 The accuracy for random forests model is 0.9904289864980346
 The Recall_score for random forests model is 0.9903294228469653
 The F1_score for random forests model is 0.9904325023867102
 The roc auc score for random forests model is 0.999919685674305


In [60]:
# confusion matrix for random forests model
print(confusion_matrix(y_test,y_test_rf))

[[1592    0    0    0    0   12    0    0    0    0    0]
 [   0 1671    1    0    0    0    0    0    0   12    1]
 [   0    0 1521    0   20    0    0    0    0    0    0]
 [   0    0    4 1555    2    0    0    0    0    0    0]
 [   0    0   10    2 1480    0    0   34    0    0    0]
 [  15    0    0    0    0 1630    0    0    6    0    0]
 [   0    0    0    0    0    0 1590    0    0    0    0]
 [   0    0    0    4   28    0    0 1553    0    0    0]
 [   0    2    0    0    0    5    0    2 1556    3    0]
 [   0    4    0    0    0    1    0    0    0 1630    0]
 [   0    0    0    0    0    0    0    0    0    0 1607]]


In [61]:
# classification report for random forests model
print(classification_report(y_test,y_test_rf))

              precision    recall  f1-score   support

           1       0.99      0.99      0.99      1604
           2       1.00      0.99      0.99      1685
           3       0.99      0.99      0.99      1541
           4       1.00      1.00      1.00      1561
           5       0.97      0.97      0.97      1526
           6       0.99      0.99      0.99      1651
           7       1.00      1.00      1.00      1590
           8       0.98      0.98      0.98      1585
           9       1.00      0.99      0.99      1568
          10       0.99      1.00      0.99      1635
          11       1.00      1.00      1.00      1607

    accuracy                           0.99     17553
   macro avg       0.99      0.99      0.99     17553
weighted avg       0.99      0.99      0.99     17553



## Decision Trees test data results 

In [62]:
y_test_dt = dt.predict(X_test)
y_test_prob_dt = dt.predict_proba(X_test)
print(f" The accuracy for decision trees model is {accuracy_score(y_test,y_test_dt)}")
print(f" The Recall_score for decision trees model is {recall_score(y_test,y_test_dt,average = 'macro')}")
print(f" The F1_score for decision trees model is {f1_score(y_test,y_test_dt,average = 'weighted')}")
print(f" The roc auc score for decision trees model is {roc_auc_score(y_test,y_test_prob_dt,average='weighted',multi_class='ovr')}")      

 The accuracy for decision trees model is 0.9741924457357717
 The Recall_score for decision trees model is 0.973981783438544
 The F1_score for decision trees model is 0.9742507800604092
 The roc auc score for decision trees model is 0.9970749762738759


In [66]:
# confusion matrix for decision trees model for test data
print(confusion_matrix(y_test,y_test_dt))

[[1555    0    0    0    0   46    0    0    3    0    0]
 [   0 1656    0    0    1    0    0    0    2   25    1]
 [   0    0 1504    0   37    0    0    0    0    0    0]
 [   0    0   23 1534    4    0    0    0    0    0    0]
 [   0    0   20    3 1425    0    0   78    0    0    0]
 [  14    0    0    0    0 1597    0    0   40    0    0]
 [   0    2    0    0    0    0 1588    0    0    0    0]
 [   0    0    1    2   93    0    0 1489    0    0    0]
 [   1    2    1    0    0   28    0    2 1532    2    0]
 [   0   21    0    0    0    0    0    0    1 1613    0]
 [   0    0    0    0    0    0    0    0    0    0 1607]]


In [65]:
# classification report for decision trees model for test data
print(classification_report(y_test,y_test_dt))

              precision    recall  f1-score   support

           1       0.99      0.97      0.98      1604
           2       0.99      0.98      0.98      1685
           3       0.97      0.98      0.97      1541
           4       1.00      0.98      0.99      1561
           5       0.91      0.93      0.92      1526
           6       0.96      0.97      0.96      1651
           7       1.00      1.00      1.00      1590
           8       0.95      0.94      0.94      1585
           9       0.97      0.98      0.97      1568
          10       0.98      0.99      0.99      1635
          11       1.00      1.00      1.00      1607

    accuracy                           0.97     17553
   macro avg       0.97      0.97      0.97     17553
weighted avg       0.97      0.97      0.97     17553



# Base line model - Most frequent  strategy

In [ ]:
# The most frequent occuring  strategy always predicts the most frequent class in the training set

In [68]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
# Train the classifier on the training set
dummy_clf.fit(X_train, y_train)
# Predict on the test set
y_test_base = dummy_clf.predict(X_test)
# predict prob on test set
y_proba = dummy_clf.predict_proba(X_test)


In [69]:
print(f" The accuracy for baseline model is {accuracy_score(y_test,y_test_base)}")
print(f" The Recall_score for baseline model is {recall_score(y_test,y_test_base,average = 'macro')}")
print(f" The F1_score for baseline model is {f1_score(y_test,y_test_base,average = 'weighted')}")
print(f" The roc auc score for baseline model is {roc_auc_score(y_test,y_proba,average='weighted',multi_class='ovr')}")      

 The accuracy for baseline model is 0.08693670597618641
 The Recall_score for baseline model is 0.09090909090909091
 The F1_score for baseline model is 0.013906956687421819
 The roc auc score for baseline model is 0.5


In [70]:
# confusion matrix for base model
print(confusion_matrix(y_test,y_test_base))

[[   0    0    0    0 1604    0    0    0    0    0    0]
 [   0    0    0    0 1685    0    0    0    0    0    0]
 [   0    0    0    0 1541    0    0    0    0    0    0]
 [   0    0    0    0 1561    0    0    0    0    0    0]
 [   0    0    0    0 1526    0    0    0    0    0    0]
 [   0    0    0    0 1651    0    0    0    0    0    0]
 [   0    0    0    0 1590    0    0    0    0    0    0]
 [   0    0    0    0 1585    0    0    0    0    0    0]
 [   0    0    0    0 1568    0    0    0    0    0    0]
 [   0    0    0    0 1635    0    0    0    0    0    0]
 [   0    0    0    0 1607    0    0    0    0    0    0]]


In [71]:
# classification report for base model
print(classification_report(y_test,y_test_base))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00      1604
           2       0.00      0.00      0.00      1685
           3       0.00      0.00      0.00      1541
           4       0.00      0.00      0.00      1561
           5       0.09      1.00      0.16      1526
           6       0.00      0.00      0.00      1651
           7       0.00      0.00      0.00      1590
           8       0.00      0.00      0.00      1585
           9       0.00      0.00      0.00      1568
          10       0.00      0.00      0.00      1635
          11       0.00      0.00      0.00      1607

    accuracy                           0.09     17553
   macro avg       0.01      0.09      0.01     17553
weighted avg       0.01      0.09      0.01     17553



C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasuk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# Base line model - stratified 

In [ ]:
# The stratified strategy predicts class probabilities based on the class distribution in the training set.

In [72]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")
# Train the classifier on the training set
dummy_clf.fit(X_train, y_train)
# Predict on the test set
y_test_str = dummy_clf.predict(X_test)
# predict prob on test set
y_proba_str = dummy_clf.predict_proba(X_test)


In [73]:
print(f" The accuracy for baseline model is {accuracy_score(y_test,y_test_str)}")
print(f" The Recall_score for baseline model is {recall_score(y_test,y_test_str,average = 'macro')}")
print(f" The F1_score for baseline model is {f1_score(y_test,y_test_str,average = 'weighted')}")
print(f" The roc auc score for baseline model is {roc_auc_score(y_test,y_proba_str,average='weighted',multi_class='ovr')}")      

 The accuracy for baseline model is 0.09149433145331283
 The Recall_score for baseline model is 0.09148552752183532
 The F1_score for baseline model is 0.09146205327993441
 The roc auc score for baseline model is 0.5000131020735387


In [74]:
# confusion matrix for stratified model
print(confusion_matrix(y_test,y_test_str))

[[136 138 176 140 148 141 157 138 147 137 146]
 [157 178 139 163 158 158 158 131 157 146 140]
 [135 135 147 140 144 118 137 140 130 149 166]
 [147 148 134 126 137 122 146 142 147 146 166]
 [142 139 140 142 153 153 140 135 120 138 124]
 [138 144 157 162 162 139 158 146 155 155 135]
 [154 128 139 130 156 140 167 140 165 143 128]
 [126 168 133 144 157 157 136 141 143 137 143]
 [125 151 147 128 146 154 128 170 141 138 140]
 [160 151 144 154 136 145 140 170 131 146 158]
 [136 165 130 146 174 139 143 142 148 152 132]]


In [75]:
# classification report for stratified model
print(classification_report(y_test,y_test_str))

              precision    recall  f1-score   support

           1       0.09      0.08      0.09      1604
           2       0.11      0.11      0.11      1685
           3       0.09      0.10      0.09      1541
           4       0.08      0.08      0.08      1561
           5       0.09      0.10      0.10      1526
           6       0.09      0.08      0.09      1651
           7       0.10      0.11      0.10      1590
           8       0.09      0.09      0.09      1585
           9       0.09      0.09      0.09      1568
          10       0.09      0.09      0.09      1635
          11       0.08      0.08      0.08      1607

    accuracy                           0.09     17553
   macro avg       0.09      0.09      0.09     17553
weighted avg       0.09      0.09      0.09     17553

